<a href="https://colab.research.google.com/github/dharpranoy/Neural-Net-GoogleColab/blob/main/nlp_embeddings_and_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word Embeddings and Sentiment**

In this colab, you'll work with word embeddings and train a basic neural network to predict text sentiment. At the end, you'll be able to visualize how the network sees the related sentiment of each word in the dataset.

## **Import TensorFlow and related functions**

In [1]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
from google.colab import drive
#drive.mount('/content/drive')

## **Get the dataset**

We're going to use a dataset containing Amazon and Yelp reviews, with their related sentiment (1 for positive, 0 for negative). This dataset was originally extracted from [here](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set).

In [4]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('/content/combined_data.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## **Tokenize the dataset**

Tokenize the dataset, including padding and OOV.

In [5]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, 
                       maxlen=max_length, 
                       padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,
                               maxlen=max_length, 
                               padding=padding_type, 
                               truncating=trunc_type)

## **Review a Sequence**

Let's quickly take a look at one of the padded sequences to ensure everything above worked appropriately.

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


It's padded a little further. There's a lot of question marks in this case you know just the 0s.

## **Train a Basic Sentiment Model with Embeddings**

In [7]:
# Build a basic sentiment network
# Note the embedding layer is first, 
# and the output is only 1 node as it is either 0 or 1 (negative or positive)
model = tf.keras.Sequential(layers=[
    tf.keras.layers.Embedding(input_dim=vocab_size,     # NEWLY ADDED
                              output_dim=embedding_dim, 
                              input_length=max_length), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=6, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 6)                 9606      
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Train the model.
num_epochs = 10
model.fit(padded, training_labels_final,
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 4s 10ms/step - loss: 0.6924 - accuracy: 0.5229 - val_loss: 0.7040 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 6ms/step - loss: 0.6856 - accuracy: 0.5304 - val_loss: 0.6968 - val_accuracy: 0.4160
Epoch 3/10
50/50 [==============================] - 0s 6ms/step - loss: 0.6618 - accuracy: 0.6798 - val_loss: 0.6708 - val_accuracy: 0.6216
Epoch 4/10
50/50 [==============================] - 0s 7ms/step - loss: 0.5940 - accuracy: 0.8186 - val_loss: 0.6168 - val_accuracy: 0.7168
Epoch 5/10
50/50 [==============================] - 0s 6ms/step - loss: 0.4886 - accuracy: 0.8763 - val_loss: 0.5531 - val_accuracy: 0.7268
Epoch 6/10
50/50 [==============================] - 0s 7ms/step - loss: 0.3781 - accuracy: 0.9077 - val_loss: 0.5299 - val_accuracy: 0.7444
Epoch 7/10
50/50 [==============================] - 0s 7ms/step - loss: 0.2976 - accuracy: 0.9328 - val_loss: 0.4969 - val_accuracy: 0.7419
Epoch 8/10
50/50 [=

## **Get files for visualizing the network**

The code below will download two files for visualizing how your network "sees" the sentiment related to each word. Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox. This will better show whether there is separation between positive and negative sentiment (or not).

In [9]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


Because we have a 1000 possibe words, the weight shape is (1000, 16).

In [10]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8') # vector
out_m = io.open('meta.tsv', 'w', encoding='utf-8') # metadata

# Write out every single word in the word index
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
  
out_v.close()
out_m.close()

In [11]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Predicting Sentiment in New Reviews**

Now that you've trained and visualized your network, take a look below at how we can predict sentiment in new reviews the network has never seen before.

In [12]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.9748191]


I hate spaghetti
[0.06881921]


Everything was cold
[0.29244098]


Everything was hot exactly as I wanted
[0.64605045]


Everything was green
[0.46053195]


the host seated us immediately
[0.7635149]


they gave us free chocolate cake
[0.78357524]


not sure about the wilted flowers on the table
[0.02511938]


only works when I stand on tippy toes
[0.81523806]


does not work when I stand on my head
[0.00981406]


